In [6]:
import pandas as pd
import numpy as np
import yfinance as yf
from apscheduler.schedulers.blocking import BlockingScheduler
from oandapyV20 import API
import oandapyV20.endpoints.orders as orders
from oandapyV20.contrib.requests import MarketOrderRequest
from oanda_candles import Pair, Gran, CandleCollector, CandleClient
from oandapyV20.contrib.requests import TakeProfitDetails, StopLossDetails
from config import access_token, accountID

data = yf.download('EURUSD=X', start='2000-01-01')
data = data[['Close']]
data = data.dropna()

half_life = 25248  # From mean-reversion-and-stationarity.ipynb

[*********************100%%**********************]  1 of 1 completed


In [7]:
data['MovingAvg'] = data['Close'].rolling(window=half_life).mean()
data['MovingStd'] = data['Close'].rolling(window=half_life).std()
data['Z'] = (data['Close'] - data['MovingAvg']) / data['MovingStd']
data['Position'] = -data['Z']
data['PnL'] = data['Position'].shift(1) * (data['Close'].diff())
data['Cumulative PnL'] = data['PnL'].cumsum()

In [8]:
def get_candles(n):
    client = CandleClient(access_token, real=False)
    collector = client.get_collector(Pair.EUR_USD, Gran.M15)
    candles = collector.grab(n)
    return candles

In [9]:
def trading_job():
    # Fetching latest price data
    candles = get_candles(half_life) 
    dfstream = pd.DataFrame(columns=['Open', 'Close', 'High', "Low"])

    # Parsing candle data for trading signals
    for i, candle in enumerate(candles):
        dfstream.loc[i] = [
            float(candle.bid.o.value),  
            float(candle.bid.c.value),
            float(candle.bid.h.value),
            float(candle.bid.l.value)
        ]

    # Calculate the rolling mean and standard deviation for the z-score
    dfstream['MovingAvg'] = dfstream['Close'].rolling(window=672).mean()  
    dfstream['MovingStd'] = dfstream['Close'].rolling(window=672).std()
    dfstream['Z'] = (dfstream['Close'] - dfstream['MovingAvg']) / dfstream['MovingStd']

    # Dropping the initial rows where the rolling window is not full
    dfstream.dropna(inplace=True)

    # Generate signals based on the z-score
    dfstream['Signal'] = 0  # Initialize all signals to 0
    dfstream.loc[dfstream['Z'] > 1, 'Signal'] = 1   # Sell signal
    dfstream.loc[dfstream['Z'] < -1, 'Signal'] = 2  # Buy signal

    last_signal = dfstream['Signal'].iloc[-1]

    client = API(access_token)

    SLTPRatio = 2
    previous_candleR = abs(dfstream['Open'].iloc[-2] - dfstream['Close'].iloc[-2])

    SLBuy = float(str(candle.bid.o))-previous_candleR
    SLSell = float(str(candle.bid.o))+previous_candleR

    TPBuy = float(str(candle.bid.o))+previous_candleR*SLTPRatio
    TPSell = float(str(candle.bid.o))-previous_candleR*SLTPRatio

    print(dfstream.iloc[:-1, :])
    print(TPBuy, " ", SLBuy, " ", TPSell, " ", SLSell)

    # Sell
    if last_signal == 1:
        mo = MarketOrderRequest(instrument="EUR_USD", units=-10000, takeProfitOnFill=TakeProfitDetails(price=TPSell).data, stopLossOnFill=StopLossDetails(price=SLSell).data)
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        print(rv)

    # Buy
    elif last_signal == 2:
        mo = MarketOrderRequest(instrument="EUR_USD", units=10000, takeProfitOnFill=TakeProfitDetails(price=TPBuy).data, stopLossOnFill=StopLossDetails(price=SLBuy).data)
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        print(rv)

In [10]:
# MANUAL
trading_job()

# SCHEDULED
#scheduler = BlockingScheduler()
#scheduler.add_job(trading_job, 'cron', day_of_week='mon-sun', hour='00-23', minute='0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59', second='0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59', start_date='2023-09-26 01:00:00')
#scheduler.start()

          Open    Close     High      Low  MovingAvg  MovingStd         Z  \
671    1.07356  1.07348  1.07362  1.07341   1.072709   0.003089  0.249651   
672    1.07357  1.07327  1.07361  1.07317   1.072701   0.003081  0.184609   
673    1.07347  1.07347  1.07357  1.07328   1.072693   0.003072  0.252806   
674    1.07347  1.07336  1.07347  1.07318   1.072685   0.003064  0.220163   
675    1.07338  1.07333  1.07348  1.07330   1.072677   0.003055  0.213629   
...        ...      ...      ...      ...        ...        ...       ...   
25242  1.07843  1.07847  1.07848  1.07833   1.078419   0.004039  0.012741   
25243  1.07848  1.07858  1.07874  1.07847   1.078415   0.004038  0.040759   
25244  1.07857  1.07869  1.07870  1.07852   1.078413   0.004037  0.068699   
25245  1.07870  1.07873  1.07878  1.07864   1.078409   0.004036  0.079519   
25246  1.07874  1.07853  1.07874  1.07848   1.078405   0.004034  0.031011   

       Signal  
671         0  
672         0  
673         0  
674        